In [1]:
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /drive


In [0]:
import os
os.chdir('/drive/My Drive/')

In [3]:
import numpy as np
import pandas as pd
import nltk
from nltk import word_tokenize, sent_tokenize, pos_tag #tokenization # Pos Tag
from nltk.stem import WordNetLemmatizer #Lemmatization
from nltk.corpus import wordnet,stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
chatbot = pd.read_excel('./Deep Chatbot/data.xlsx')

In [50]:
chatbot.head()

,Questions,Answers,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,anything not understood,Please enter a valid query.,NaN,NaN,NaN
1,What are the prerequisites to learn Data Scien...,There are no prerequisites for learning this c...,NaN,NaN,NaN
2,What are the prerequisites to learn Data Science?,There are no prerequisites for learning this c...,NaN,NaN,NaN
3,prerequisites to learn data science,There are no prerequisites for learning this c...,NaN,NaN,NaN
4,data science prerequisites,There are no prerequisites for learning this c...,NaN,NaN,NaN


In [0]:
chatbot = chatbot[['Questions','Answers']]

In [0]:
chatbot['Answers'].value_counts()

In [54]:
chatbot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1575 entries, 0 to 1574
Data columns (total 2 columns):
Questions    1575 non-null object
Answers      1575 non-null object
dtypes: object(2)
memory usage: 24.7+ KB


# POS TAGGING

In [0]:
def postag(pos):
    if pos.startswith('N'):
        wp = wordnet.NOUN
    elif pos.startswith('V'):
        wp = wordnet.VERB
    elif pos.startswith('R'):
        wp = wordnet.ADV
    elif pos.startswith('J'):
        wp = wordnet.ADJ
    else:
        wp = wordnet.NOUN
    
    return wp

# Text Preprocessing

In [0]:
wnl = WordNetLemmatizer()

def textprocess(doc):
    
    # Step1 : Converting into lower case
    doc = doc.lower()
    # Step2 : Remove special characters
    doc = re.sub('[^a-z]',' ',doc)
    #Step3 : pos tagging (parts of speech)
    token = word_tokenize(doc) # tokenization -get the words
    token_pos = pos_tag(token)
    # step4 : Lemma and remove stopwords
    lemma = [wnl.lemmatize(word,pos=postag(pos)) for word,pos in token_pos]
    clean = ' '.join(lemma)
    return clean

# Questions Preprocessing

In [0]:
Total_Questions1 = {}

In [0]:
for i,val in enumerate(list(chatbot['Questions'])):
  Total_Questions1.update({i:textprocess(val)})

# Cosine Similarity

In [0]:
def cosine(a,b):
    moda = np.linalg.norm(a) # magnitude of a
    modb = np.linalg.norm(b) # magnitude of b
    dotprod = np.dot(a,b) # dot product of vector a and vector b
    # a[0], b[0] -> remove shape in it, we don't want vector to have some shape
    # i.e., neither column matrix nor row matrix
    cos = dotprod/(moda*modb)
    return cos

# Question Mapping

In [0]:
def question_mapping(query): 
  query = textprocess(query)
  subject = query.split()
  documents = []
  for i in range(len(subject)):
    if i == len(subject)-1:
      pass
    else:
      documents.append(chatbot['Questions'][chatbot['Questions'].str.contains((subject[i]),case=False)])
  corpus1 = []
  for docs in documents:
    doc = list(docs)
    for dc in doc:
      corpus1.append(textprocess(dc))
      #print(corpus1)
  return corpus1

# TFIDF Vectorizer

In [0]:
def tfidf_vec(text): 
  if text:
    X = tfidf.fit_transform(text).toarray()
    return X
  else:
    text = ['enter any other valid question']
    X = tfidf.fit_transform(text).toarray()
    return X

# Chat Function

In [0]:
def chatanswers(query):
    #step-1: text processing
    clean = textprocess(query)
    #step-2: Question Mapping
    corpus = question_mapping(query) 
    Y = tfidf_vec(corpus)
    b = tfidf.transform([query]).toarray() # query in list
    cosvalue = {}
    for i,vector in enumerate(Y):
      cos = cosine(vector,b[0]) # b[0] -> remove shape in it
      if cos>0.5:
        cosvalue.update({i:cos}) # append values in dictionary
    sort = sorted(cosvalue.items(),key=operator.itemgetter(1),reverse=True)
    ind = [index for index,cosv in sort[:1]]
    var = 0
    for i in ind:
      var = i
    question_number = 0
    for key,value in Total_Questions1.items():
      if not corpus:
        question_number = 0
      elif corpus[var] in value:
        question_number = key
    return ind,list(chatbot.loc[[question_number]]['Answers'])

# Test Cases

In [0]:
def testing():
  greetings = ['hi','hii','hello','helo','hey','hey!','wassup','what is up',"what's up",'whatsup', 'whatszap','greetings','dude','what is your name?',"what is your name","what's your name?","what's your name",'your name',]
  sendoff = ['ok bye','okay bye','bye','see you','see u','good bye','byee','thank you','tc','bye!','quit','exit','ok thanks','thanku','thanks']
  while True:
    query = input("User:")
    query = textprocess(query)
    if  len(query.split()) <= 1 and (query not in greetings and query not in sendoff):
      print("KASPER : ""I’m sorry, I don’t understand.")
    elif query in greetings:
      print("KASPER : ""Hello! This is KASPER how may I help you")
    elif query in sendoff:
      print("KASPER : ""Have a nice day")
      break
    else:
      index,ans = chatanswers(query)
      if ans == []:
        print("KASPER : ""I’m sorry, I don’t understand.")
      else:
        
        for result in ans:
          print("KASPER : ",result)

In [66]:
testing()

User:what is innomatics good for?
KASPER :  Innomatics is good at both practical and theoritical methodologies taught by the trainers and providing mentorship during internships.
User:do you provide internship
KASPER :  Yes, we do provide Internship and duration will be a minimum of 3 - 6months and it completely based on your expertise.
User:bye
KASPER : Have a nice day


In [0]:
import pickle

In [0]:
pickle_out = open("chatdata.chbt","wb")
pickle.dump(chatbot, pickle_out)
pickle_out.close()

In [0]:
with open('./data.avi','rb') as f:
  

In [46]:
pickle.load('./data.avi',)

TypeError: ignored

In [0]:
textprocess('avinash gunda')

NameError: ignored

In [0]:
question_mapping('Enter a valid question')

In [0]:
chatanswers('avinash gunda')

In [0]:
a = 'data science'

In [0]:
pos_tag(a.split())